In [7]:
# 导入库

import numpy as np
import pandas as pd
import datetime

In [9]:
# 数据清理 （该代码块仅为演示作用）

file = 'raw/changqing2023.csv'
df = pd.read_csv(file)

# 重命名列
df.rename(columns={
    '月份': 'ContractMonth',
    '序号': 'ContractID',
    '合同日期': 'ContractDate',
    '交货时间' : 'DeliveryData',
    '单位': 'SalesUnitName',
    '油品规格': 'ProductName',
    '合同数量': 'ContractQuantity',
    '余量': 'ContractRemaining',
    '发出量': 'ShippedQuantity',
    '船名': 'ShipName',
    '装货地': 'LoadingAddress',
    '装船日期': 'BoardingData',
    '合同单价': 'SalePrice',
    '单位.1': 'PurchaseUnitName',
    '合同单价.1': 'PurchasePrice'
}, inplace = True)

# 删除多余列
columns_to_drop = [
    '序号.1',
    '合同日期.1',
    '交货时间.1',
    '油品规格.1',
    '合同数量.1',
    '余量.1',
    '发出量.1',
    '船名.1',
    '装货地.1',
    '装船日期.1',]

columns_to_drop_1 = [
    '交货时间.1'
]

df.drop(columns=columns_to_drop_1, inplace=True)


# 日期格式转制
def convert_chinese_date(chinese_date):
    # 将中文日期转换为标准日期格式
    date_obj = datetime.datetime.strptime(chinese_date, "%m月%d日")
    # 更改年份信息
    date_obj = date_obj.replace(year=2023) #在此处更改
    return date_obj.strftime("%Y-%m-%d")

df['ContractDate'] = df['ContractDate'].apply(convert_chinese_date)
df['DeliveryData'] = df['DeliveryData'].apply(convert_chinese_date)
df['BoardingData'] = df['BoardingData'].apply(convert_chinese_date)


# 补充空值
df.fillna({
    'ContractRemaining': 0,  # 示例：将ContractRemaining的NaN值填充为0
    # 可以根据需要添加更多的默认值填充
}, inplace=True)

df.info()

In [ ]:
# 生成合同代码

# 生成唯一ContractID
def generate_unique_contract_id(row):
    unique_string = f"{row['ContractDate']}-{row['SalesUnitID']}-{row['PurchaseUnitID']}"
    # 使用MD5哈希生成唯一ID并取其前6位，保证唯一性
    unique_id = md5(unique_string.encode()).hexdigest()[:6]
    # 取日期的后两位作为年份标识，组合成8位ID
    year_suffix = row['ContractDate'][-2:]
    return f"{year_suffix}{unique_id}"

# 创建一个新的列用于存储唯一的ContractID
df['ContractID'] = df.apply(generate_unique_contract_id, axis=1)

# 确保所有ContractID都是8位数
print(df[['ContractID']])

In [ ]:
# 链接数据库

# 连接到MySQL数据库
db = mysql.connector.connect(
    host="rm-8vbmvev6dr60510n20o.rwlb.zhangbei.rds.aliyuncs.com",
    user="administrator",
    password="Evrald520!",
    database="company_data"
)
cursor = db.cursor()

# # 关闭连接 （使用完成后再关闭连接，此处仅为示意）
# cursor.close()
# db.close()

In [ ]:
# 插入数据到companies表
companies = pd.concat([df[['SalesUnitID', 'SalesUnitName']], df[['PurchaseUnitID', 'PurchaseUnitName']].rename(columns={'PurchaseUnitID': 'SalesUnitID', 'PurchaseUnitName': 'SalesUnitName'})]).drop_duplicates()
for index, row in companies.iterrows():
    cursor.execute("INSERT INTO companies (CompanyID, CompanyName) VALUES (%s, %s) ON DUPLICATE KEY UPDATE CompanyName = VALUES(CompanyName)", (row['SalesUnitID'], row['SalesUnitName']))

# 插入数据到products表
products = df[['ProductID', 'ProductName']].drop_duplicates()
for index, row in products.iterrows():
    cursor.execute("INSERT INTO products (ProductID, ProductName) VALUES (%s, %s) ON DUPLICATE KEY UPDATE ProductName = VALUES(ProductName)", (row['ProductID'], row['ProductName']))

# 插入数据到contracts表
contracts = df[['ContractID', 'ContractMonth', 'ContractDate', 'SalesUnitID', 'PurchaseUnitID']].drop_duplicates()
for index, row in contracts.iterrows():
    cursor.execute("""
        INSERT INTO contracts (ContractID, ContractMonth, ContractDate, SalesUnitID, PurchaseUnitID)
        VALUES (%s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE ContractMonth=VALUES(ContractMonth), ContractDate=VALUES(ContractDate), SalesUnitID=VALUES(SalesUnitID), PurchaseUnitID=VALUES(PurchaseUnitID)
    """, (row['ContractID'], row['ContractMonth'], row['ContractDate'], row['SalesUnitID'], row['PurchaseUnitID']))

# 插入数据到transactions表
transactions = df[['TransactionID', 'ProductID', 'DeliveryData', 'ContractQuantity', 'ContractRemaining', 
                   'ShippedQuantity', 'ShipName', 'LoadingAddress', 'SalePrice', 'PurchasePrice']]
for index, row in transactions.iterrows():
    cursor.execute("""
        INSERT INTO transactions (TransactionID, ProductID, DeliveryDate, ContractQuantity, ContractRemaining, 
                                  ShippedQuantity, ShipName, LoadingAddress, SalePrice, PurchasePrice)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (row['TransactionID'], row['ProductID'], row['DeliveryData'], row['ContractQuantity'], row['ContractRemaining'], 
          row['ShippedQuantity'], row['ShipName'], row['LoadingAddress'], row['SalePrice'], row['PurchasePrice']))

# 插入数据到contracttransactions表
contracttransactions = df[['ContractID', 'TransactionID']]
for index, row in contracttransactions.iterrows():
    cursor.execute("INSERT INTO contracttransactions (ContractID, TransactionID) VALUES (%s, %s)", (row['ContractID'], row['TransactionID']))

# 提交事务
db.commit()

# 关闭连接
cursor.close()
db.close()